In [1]:
import pandas as pd

In [2]:
adw_camp_df = pd.read_csv('../raw_data/adwords_campaign.csv', parse_dates=['Day'], thousands=',')

In [3]:
adw_camp_df['Cost'] = adw_camp_df['Cost'].apply(lambda item: item.replace('₹','').replace(',',''))\
                                         .astype(float)

In [4]:
adw_conv = pd.read_csv('../raw_data/adwords_conversions.csv', parse_dates=['Day'], thousands=',')

In [5]:
adw_conv['Conversions'] = adw_conv['Conversions'].astype('int64')

In [6]:
reorder_col = ['Day', 'Campaign', 'Device', 'Conversion action', 'Conversions']

In [7]:
adw_conv = adw_conv[reorder_col].sort_values(by=['Day', 'Campaign', 'Conversions'], ascending = [True, True, False])

In [9]:
adw_conv_df = adw_conv.set_index(keys=['Day', 'Campaign', 'Device', 'Conversion action'])\
                      .unstack(fill_value=0)\
                      .reset_index()

In [10]:
adw_conv_df.columns =  [''.join(col).strip().replace('Conversions', '') for col in adw_conv_df.columns.values]

In [11]:
adw_conv = adw_conv_df

In [12]:
adw_camp_conv_df = adw_camp_df.merge(adw_conv,how='left',on=['Day', 'Campaign', 'Device']).fillna(0)

In [13]:
for item in adw_camp_conv_df.select_dtypes('float').columns.tolist()[1:]:
    adw_camp_conv_df[item] = adw_camp_conv_df[item].astype('int64')

In [14]:
adw_map_df = pd.read_csv('../raw_data/adwords_mapping.csv')

In [15]:
map_campaign = adw_map_df['Campaign']

In [16]:
if map_campaign.duplicated(keep='first').sum() == 0:
    print(f'There are no duplicate campaigns in the mapping file')
else:
    print(f'There are {map_campaign.duplicated(keep="first").sum()} Campaigns in the mapping file.')

There are no duplicate campaigns in the mapping file


In [17]:
adw_overall = adw_camp_conv_df.merge(adw_map_df,how='left', on=['Campaign'])

In [18]:
if adw_overall['Category'].isnull().sum() > 0:
    print('Please add the campaign names present in the adwords_mapping_new_campaigns file to the mapping file')
    new_camps = adw_overall[adw_overall['Category'].isnull()]['Campaign'].drop_duplicates()
    new_camps.to_csv('../adwords_mapping_new_campaigns.csv', index=False)
else:
    print('Status: OK')

Status: OK


In [19]:
adw_overall['Month'] = adw_overall['Day'].dt.strftime('%b-%y')

In [20]:
del adw_overall['Conversions']

In [21]:
col_index = [0,13,12,1,2,3,4,5,6,7,8,9,10,11]

In [22]:
reoder_cols = [adw_overall.columns.tolist()[i] for i in col_index]

In [23]:
adw_overall[reoder_cols].to_csv('../output/adwords_only_conversion_actions.csv', index = False)